# Aes python

 In this lesson you will learn how to use the Python library
PyCryptodome to encrypt and decrypt with the AES algorithm. This
lesson requires an understanding of Python IO, byte manipulation,
and endianess. Good luck!


### Challenge Name: aes_decrypt (/embsec/aes_python/aes_decrypt)


    The serial device is sending you a message encrypted with the key
    stored in aeskeyfile.bin. The message is in the following format:
       
    [ 0x10 ] [      0x2      ] [ variable...]
    ----------------------------------------
    |  IV   | Ciphertext Size | Ciphertext |
    ----------------------------------------
    
    1. Read the AES key from 'aeskeyfile0.bin'
    2. Read the 16 byte initialization vector (IV) from the serial device
    3. Read the ciphertext size (formatted as a little-endian short) from the serial device
    4. Read the ciphertext from the serial device
    5. Decrypt the ciphertext using 128-bit AES in CBC mode
    
Resources:

<https://docs.python.org/3/library/struct.html>

<https://pycryptodome.readthedocs.io/en/latest/src/cipher/aes.html>



In [7]:
from embsec import Serial
import struct
#import cryptodome stuff
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad

def aes_decrypt():
    ser = Serial("/embsec/aes_python/aes_decrypt")
    # Your code goes here!
    with open('aeskeyfile0.bin', 'rb') as fp:
        key = fp.read()
    
    print('spot 1')
    
    iv = ser.read(16)
    
    print('spot 2')
    
    ee = struct.unpack('<h', ser.read(2))
    ciphertext_size = ee[0]
    
    print('spot 3')
    
    ciphertext = ser.read(ciphertext_size)
    
    print('spot 4')
    
#     file_in = open("encrypted.bin", "rb")
#     nonce, tag, ciphertext = [ ciphertext.read(x) for x in (16, 16, -1) ]
    
#     print('spot 5')
    
    # let's assume that the key is somehow available again
    cipher = AES.new(key, AES.MODE_CBC, iv)
#     data = unpad(cipher.decrypt(ciphertext), AES.block_size)
    data = cipher.decrypt(ciphertext)
    print(data)
    

aes_decrypt()


spot 1
spot 2
spot 3
spot 4
b'embsec{aes_decrypt_a453e1fec88cb032}\n\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'


### Challenge Name: aes_encrypt_file (/embsec/aes_python/aes_encrypt_file)


    You need to send an encrypted message to the serial device. Encrypt
    the contents of 'plaintext.bin' with the key found in 'aeskeyfile.bin' 
    and a random IV. The message needs to be formatted as follows:
    
    [ 0x10 ] [      0x2      ] [ variable...]
    ----------------------------------------
    |  IV   | Ciphertext Size | Ciphertext |
    ----------------------------------------

    1. Read the AES key from 'aeskeyfile1.bin'
    2. Encrypt the plaintext located in 'plaintext.bin'
    3. Send the IV to the serial device
    4. Send the ciphertext size (formatted as a little-endian short) to the serial device
    5. Send the ciphertext to the serial device
    6. Read the response

Resources:

<https://docs.python.org/3/library/struct.html>

<https://pycryptodome.readthedocs.io/en/latest/src/cipher/aes.html>




In [ ]:
from embsec import Serial

def aes_encrypt_file():
    ser = Serial("/embsec/aes_python/aes_encrypt_file")
    # Your code goes here!

aes_encrypt_file()


### Challenge Name: aes_encrypted_echo (/embsec/aes_python/aes_encrypted_echo)


    The serial device is sending you an encrypted message. Decrypt
    the message and send it back encrypt with a different IV. The 
    message needs to be formatted as follows:
    
    [ 0x10 ] [      0x2      ] [ variable...]
    ----------------------------------------
    |  IV   | Ciphertext Size | Ciphertext |
    ----------------------------------------
    
    1. Read the AES key from 'aeskeyfile2.bin'
    2. Read the 16 byte initialization vector (IV) from the serial device
    3. Read the ciphertext size (formatted as a little-endian short) from the serial device
    4. Read the ciphertext from the serial device
    5. Decrypt the ciphertext using 128-bit AES in CBC mode
    2. Re-encrypt the plaintext with the same key but a new IV
    3. Send the IV to the serial device
    4. Send the ciphertext size (formatted as a little-endian short) to the serial device
    5. Send the ciphertext to the serial device
    6. Read the response




In [1]:
from embsec import Serial

def aes_encrypted_echo():
    ser = Serial("/embsec/aes_python/aes_encrypted_echo")
    # Your code goes here!

aes_encrypted_echo()


In [11]:
from Crypto.Cipher import AES
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA256
from Crypto.Util import Padding
from Crypto.Signature import pkcs1_15
from Crypto.Util.Padding import unpad
import struct
"""

"""
def protect_firmware(infile, outfile, version, message):
    with open(infile, 'rb') as f: #gets firmware
        fw = f.read()
        
    metadata = struct.pack("<HH", version, len(fw)) #packs metadata: version, length  of unencrypted firmware
    fw = fw + message.encode() + b'\00' #adds message to end of firmware
    
    #THIS SECTION (UP UNTIL THE NEXT COMMENT) WAS ADDED TO WRITE KEYS TO ANOTHER FILE. IT WAS NOT PART OF TOM'S ORIGINAL CODE
    
    key = RSA.generate(2048)
    f = open('mykey.pem','wb')
    f.write(key.export_key('PEM'))
    f.close()
    aes_key = b'Sixteen byte key'
#     with open('aes_key.bin', 'wb') as fh:
#         fh.write(b'Sixteen byte key')
    
    #END BLOCK
    
    with open("mykey.pem", 'rb') as sec_output:
#         aes_key = sec_output.read(16) #get symmetric key
        rsa_key = RSA.import_key(sec_output.read()) #get private key
        
    cipher = AES.new(aes_key, AES.MODE_CBC) #creates AES object
    
    encrypted_fw = cipher.encrypt(Padding.pad(fw, AES.block_size)) #encrypts
    
    metadata += struct.pack("<H", len(encrypted_fw)) #adds the length of encrypted firmware to metadata
    
    hashed_fw = SHA256.new(data = metadata + cipher.iv + encrypted_fw) #hashes the metadata, IV, and encrypted firmware
    
    signature = pkcs1_15.new(rsa_key).sign(hashed_fw) #signs the hashed firmware
    
    fw_blob = signature + metadata + cipher.iv + encrypted_fw #creates blob to be sent to bootloader
    print('length of iv: ', len(cipher.iv))
    
    with open(outfile, "w+b") as out: #writes firmware blob to another file
        out.write(fw_blob)
#     print('program completed')
    return 0

def undo_protect():
    with open('hello.bin', 'rb') as fh:
        stuff = fh.read()
        
    sig = stuff[:255]
    met = stuff[256:261]
    IV = stuff[262:(262 +16)]
    encrypted_fw = stuff[(262 + 16):]
    
    cipher = AES.new(b'Sixteen byte key', AES.MODE_CBC, IV)

    data = cipher.decrypt(encrypted_fw)
    print(data)


protect_firmware("test_firmware.bin", "hello.bin", 5, "this is a test to make sure that this program works")
undo_protect()

length of iv:  16
b'1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001this is a test to make sure that this program works\x00\n\n\n\n\n\n\n\n\n\n'
